In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from docx import Document
Air_db= pd.read_spss('combine.sav')

Declaring the categories and continous data

In [2]:
categorical_cols= ['SocioEconomic','EmergencyorElective01','Instrumentaldelivery','Diabethistory','Familyhistoryofdiabetes','Familyhistoryofpreeclampsia','AutoImmuneDisease',
                        'hospitalized','sex','Numberofpreviouspregnanciesgravidity','Antenatalmedicationsmaternal','Birthweight_cate','PretermGrowth_cate',
                        'FulltermGrowth_cate','AddmisionForMOrthan72HoursReason_newborn','Met1_zscore','Met2_zscore','Met3_zscore','ethnicities',
                        'Macrosomia_cate','caesarean_cate','hypertension_cate','age_cate','TermStatus1preterm2fullterm_newborn','ApparentAbnormalitiesInBewBornTime_01']

continous_cols= ['age','BMI','GestationalAge_newborn','NewBornWeight_newborn','NewBornHeightInAccouchement_newborn','NewBornHeadCircumference_newborn',
                      'NewBornChestCircumference_newborn','NewBornHipCircumference_newborn','NewBornUpperArmCircumference_newborn','AG_Size_newborn','ASD_Size_newborn',
                      'PW_Size_newborn','ACD_Size_newborn','AFD_Size_newborn','AnteriorFontanelleAnteroposteriorDiameter_newborn','AnteriorFontanelleTransverseDiameter_newborn',
                      'neonatalintensivecareunitNICUdays','Hospitaldaycount','Met_pha1_zscore','Met_pha2_zscore','Met_pha3_zscore','AirPollution_month1','AirPollution_month2','AirPollution_month3',
                      'AirPollution_meanfirstthreemonth','AirPollution_month4','AirPollution_month5','AirPollution_month6','AirPollution_meansecondthreemonth',
                      'AirPollution_month7','AirPollution_month8','AirPollution_month9','AirPollution_month10','AirPollution_meanlastfourmonth']

Air_db[categorical_cols] = Air_db[categorical_cols].astype("category")
Air_db[continous_cols] = Air_db[continous_cols].astype("float64")

In [ ]:
from scipy.stats import shapiro

for col in continous_cols:
    stat, p = shapiro(Air_db[col])
    print(f'{col}: Statistic={stat:.3f}, p-value={p:.3f}')
    if p > 0.05:
        print('Probably normal')
    else:
        print('Probably not normal')

descrptive analysis for categorical data

In [4]:
# Define target column
preterm_column = "TermStatus1preterm2fullterm_newborn"

# Drop target column from the list for testing against it
categorical_cols_for_test = [col for col in categorical_cols if col != preterm_column]

# Remove columns with only one unique non-missing value
categorical_cols_for_test = [col for col in categorical_cols_for_test if Air_db[col].dropna().nunique() > 1]

# Perform Chi-Square tests
chi2_results = []
contingency_tables = {}

for col in categorical_cols_for_test:
    # Drop rows with missing values in either the target or current column
    valid_data = Air_db[[preterm_column, col]].dropna()

    if valid_data.empty:
        print(f"Skipping {col} (All values missing)")
        continue

    # Create contingency table
    contingency_table = pd.crosstab(valid_data[preterm_column], valid_data[col])

    if contingency_table.empty:
        print(f"Skipping {col} (Empty Contingency Table)")
        continue

    # Chi-Square Test (explicitly set correction=False)
    chi2, p, dof, expected = stats.chi2_contingency(contingency_table, correction=False)

    # Row-wise percentages
    percentage_table = contingency_table.div(contingency_table.sum(axis=1), axis=0) * 100

    # Save results
    chi2_results.append({
        "Column": col,
        "Chi2": chi2,
        "P-Value": p,
        "DoF": dof
    })

    # Store tables
    contingency_tables[col] = pd.concat([contingency_table, percentage_table], keys=["Count", "Percentage"])

# Compile results
chi2_results = pd.DataFrame(chi2_results)


saving the results in Excel

In [5]:
with pd.ExcelWriter("chi2_results_with_percentages.xlsx") as writer:
    chi2_results.to_excel(writer, sheet_name="Chi2 Results", index=False)
    
    for col, table in contingency_tables.items():
        # Truncate sheet name to 31 characters
        safe_sheet_name = f"Table_{col}"[:31]
        table.to_excel(writer, sheet_name=safe_sheet_name)

descriptive analysis for continous data

In [ ]:
# Define target column
preterm_column = "TermStatus1preterm2fullterm_newborn"

# Store results
mw_test_results = []
summary_stats = {}

# Summary statistics (already good for nonparametric analysis)
for col in continous_cols:
    stats_df = Air_db.groupby(preterm_column)[col].agg(
        mean="mean",
        std="std",
        count="count",
        min="min",
        median="median",
        max="max",
        Q1=lambda x: x.quantile(0.25),
        Q3=lambda x: x.quantile(0.75)
    )

    stats_df["IQR"] = stats_df["Q3"] - stats_df["Q1"]
    summary_stats[col] = stats_df.drop(columns=["count"])

# Combine summary tables
summary_df = pd.concat(summary_stats, axis=1)

# Explicit group labels (SPSS-style: 1 vs 2)
group_labels = sorted(Air_db[preterm_column].dropna().unique())
group1_label = group_labels[0]
group2_label = group_labels[1]

# Perform Mann–Whitney U test
for col in continous_cols:
    group1 = Air_db.loc[Air_db[preterm_column] == group1_label, col].dropna()
    group2 = Air_db.loc[Air_db[preterm_column] == group2_label, col].dropna()

    if len(group1) > 1 and len(group2) > 1:
        u_stat, p_value = stats.mannwhitneyu(
            group1,
            group2,
            alternative="two-sided"
        )

        mw_test_results.append({
            "Variable": col,
            "U-Statistic": u_stat,
            "P-Value": p_value,
            f"Median_{group1_label}": group1.median(),
            f"Median_{group2_label}": group2.median(),
        })
    else:
        print(f"Skipping {col} (Not enough data in one or both groups)")

# Convert results to DataFrame
mw_test_df = pd.DataFrame(mw_test_results)


Saving the result in an excel

In [10]:
with pd.ExcelWriter("statistical_results.xlsx") as writer:
    summary_df.to_excel(writer, sheet_name="Summary Statistics")
    mw_test_df.to_excel(writer, sheet_name="T-Test Results")

print("Results saved to 'statistical_results.xlsx'")

Results saved to 'statistical_results.xlsx'


Ordinal logistic Regression

In [ ]:
# Define the air pollution columns
pollution_cols = [
    "AirPollution_meanfirstthreemonth",
    "AirPollution_meansecondthreemonth",
    "AirPollution_meanlastfourmonth"
]

# Loop over each column to create dummies
for i, col in enumerate(pollution_cols, start=1):
    q75 = Air_db[col].quantile(0.75)
    dummy_col = f"AirPollution{i}_Dummy"
    Air_db[dummy_col] = (Air_db[col] >= q75).astype('category')

Crude Model

In [ ]:
###### Ensure y is numeric
y = pd.to_numeric(Air_db['Birthweight_cate'], errors='coerce')
#y = pd.to_numeric(Air_db['PretermGrowth_cate'], errors='coerce')
#y = pd.to_numeric(Air_db['FulltermGrowth_cate'], errors='coerce')

###### Dummy encode binary categorical variables
X = pd.get_dummies(Air_db[['AirPollution1_Dummy']], drop_first=True)
#X = pd.get_dummies(Air_db[['AirPollution2_Dummy']], drop_first=True)
#X = pd.get_dummies(Air_db[['AirPollution3_Dummy']], drop_first=True)


###### Convert all to float (fix for bools)
X = X.astype(float)

###### Fit the model
from statsmodels.miscmodels.ordinal_model import OrderedModel
model = OrderedModel(y, X, distr='logit', missing='drop')
result = model.fit(method='bfgs', maxiter=1000, disp=True)
print(result.summary())

###### Odds Ratio
coef = result.params
odds_ratios = np.exp(coef)
conf = result.conf_int()
conf_odds = np.exp(conf)
print("\nOdds Ratio and 95% CI for AirPollution1_Dummy_True:")
print(f"OR: {odds_ratios['AirPollution1_Dummy_True']}")
print(f"95% CI: ({conf_odds.loc['AirPollution1_Dummy_True', 0]}, {conf_odds.loc['AirPollution1_Dummy_True', 1]})")

Model one

In [ ]:
###### Ensure y is numeric
y = pd.to_numeric(Air_db['Birthweight_cate'], errors='coerce')
#y = pd.to_numeric(Air_db['PretermGrowth_cate'], errors='coerce')
#y = pd.to_numeric(Air_db['FulltermGrowth_cate'], errors='coerce')

###### Dummy encode binary categorical variables
X_cat = pd.get_dummies(Air_db[['AirPollution1_Dummy', 'sex']], drop_first=True)
#X_cat = pd.get_dummies(Air_db[['AirPollution2_Dummy', 'sex']], drop_first=True)
#X_cat = pd.get_dummies(Air_db[['AirPollution3_Dummy', 'sex']], drop_first=True)

###### Continuous variables
X_cont = Air_db[['age', 'BMI']]

###### Combine predictors
X = pd.concat([X_cat, X_cont], axis=1)

###### Convert all to float (fix for bools)
X = X.astype(float)

###### Fit the model
from statsmodels.miscmodels.ordinal_model import OrderedModel
model = OrderedModel(y, X, distr='logit', missing='drop')
result = model.fit(method='bfgs', maxiter=1000, disp=True)
print(result.summary())

###### Odds Ratio
coef = result.params
odds_ratios = np.exp(coef)
conf = result.conf_int()
conf_odds = np.exp(conf)
print("\nOdds Ratio and 95% CI for AirPollution1_Dummy_True:")
print(f"OR: {odds_ratios['AirPollution1_Dummy_True']}")
print(f"95% CI: ({conf_odds.loc['AirPollution1_Dummy_True', 0]}, {conf_odds.loc['AirPollution1_Dummy_True', 1]})")

Model two

In [ ]:
###### Ensure y is numeric
y = pd.to_numeric(Air_db['Birthweight_cate'], errors='coerce')
#y = pd.to_numeric(Air_db['PretermGrowth_cate'], errors='coerce')
#y = pd.to_numeric(Air_db['FulltermGrowth_cate'], errors='coerce')

###### Dummy encode binary categorical variables
X_cat = pd.get_dummies(Air_db[['AirPollution1_Dummy', 'sex']], drop_first=True)
#X_cat = pd.get_dummies(Air_db[['AirPollution2_Dummy', 'sex']], drop_first=True)
#X_cat = pd.get_dummies(Air_db[['AirPollution3_Dummy', 'sex']], drop_first=True)

###### Other variables
X_oth = Air_db[['age', 'BMI','SocioEconomic','Met_pha1_zscore','ethnicities']]
#X_oth = Air_db[['age', 'BMI','SocioEconomic','Met_pha2_zscore','ethnicities']]
#X_oth = Air_db[['age', 'BMI','SocioEconomic','Met_pha3_zscore','ethnicities']]

###### Combine predictors
X = pd.concat([X_cat, X_oth], axis=1)

###### Convert all to float (fix for bools)
X = X.astype(float)

###### Fit the model
from statsmodels.miscmodels.ordinal_model import OrderedModel
model = OrderedModel(y, X, distr='logit', missing='drop')
result = model.fit(method='bfgs', maxiter=1000, disp=True)
print(result.summary())

###### Odds Ratio
coef = result.params
odds_ratios = np.exp(coef)
conf = result.conf_int()
conf_odds = np.exp(conf)
print("\nOdds Ratio and 95% CI for AirPollution1_Dummy_True:")
print(f"OR: {odds_ratios['AirPollution1_Dummy_True']}")
print(f"95% CI: ({conf_odds.loc['AirPollution1_Dummy_True', 0]}, {conf_odds.loc['AirPollution1_Dummy_True', 1]})")

Third Model

In [ ]:

###### Ensure y is numeric
y = pd.to_numeric(Air_db['Birthweight_cate'], errors='coerce')
#y = pd.to_numeric(Air_db['PretermGrowth_cate'], errors='coerce')
#y = pd.to_numeric(Air_db['FulltermGrowth_cate'], errors='coerce')

###### Dummy encode binary categorical variables
X_cat = pd.get_dummies(Air_db[['AirPollution1_Dummy', 'sex','ApparentAbnormalitiesInBewBornTime_01','EmergencyorElective01']], drop_first=True)
#X_cat = pd.get_dummies(Air_db[['AirPollution2_Dummy', 'sex','ApparentAbnormalitiesInBewBornTime_01','EmergencyorElective01']], drop_first=True)
#X_cat = pd.get_dummies(Air_db[['AirPollution3_Dummy', 'sex','ApparentAbnormalitiesInBewBornTime_01','EmergencyorElective01']], drop_first=True)

###### Other variables
X_oth = Air_db[['age', 'BMI','SocioEconomic','Met_pha1_zscore','ethnicities']]
#X_oth = Air_db[['age', 'BMI','SocioEconomic','Met_pha2_zscore','ethnicities']]
#X_oth = Air_db[['age', 'BMI','SocioEconomic','Met_pha3_zscore','ethnicities']]


###### Combine predictors
X = pd.concat([X_cat, X_oth], axis=1)

###### Convert all to float (fix for bools)
X = X.astype(float)

###### Fit the model
from statsmodels.miscmodels.ordinal_model import OrderedModel
model = OrderedModel(y, X, distr='logit', missing='drop')
result = model.fit(method='bfgs', maxiter=1000, disp=True)
print(result.summary())

###### Odds Ratio
coef = result.params
odds_ratios = np.exp(coef)
conf = result.conf_int()
conf_odds = np.exp(conf)
print("\nOdds Ratio and 95% CI for AirPollution1_Dummy_True:")
print(f"OR: {odds_ratios['AirPollution1_Dummy_True']}")
print(f"95% CI: ({conf_odds.loc['AirPollution1_Dummy_True', 0]}, {conf_odds.loc['AirPollution1_Dummy_True', 1]})")

Ordinal Logistic Regression by Cold and Warm Seasons

Crude Model

In [ ]:
# Loop over each season
for season, df_season in Air_db.groupby('season'):
    print(f"\n\n####### Results for Season: {season} #######\n")
    ###### Ensure y is numeric
    y = pd.to_numeric(df_season['Birthweight_cate'], errors='coerce')
    #y = pd.to_numeric(df_season['PretermGrowth_cate'], errors='coerce')
    #y = pd.to_numeric(df_season['FulltermGrowth_cate'], errors='coerce')

    ###### Dummy encode binary categorical variables
    X = pd.get_dummies(df_season[['AirPollution1_Dummy']], drop_first=True)
    #X = pd.get_dummies(df_season[['AirPollution2_Dummy']], drop_first=True)
    #X = pd.get_dummies(df_season[['AirPollution3_Dummy']], drop_first=True)


    ###### Convert all to float (fix for bools)
    X = X.astype(float)

    ###### Fit the model
    from statsmodels.miscmodels.ordinal_model import OrderedModel
    model = OrderedModel(y, X, distr='logit', missing='drop')

    try:
        result = model.fit(method='bfgs', maxiter=1000, disp=False)
        print(result.summary())
    except Exception as e:
        print(f"Model fitting failed for season {season}: {e}")

    ###### Odds Ratio
    coef = result.params
    odds_ratios = np.exp(coef)
    conf = result.conf_int()
    conf_odds = np.exp(conf)
    print("\nOdds Ratio and 95% CI for AirPollution1_Dummy_True:")
    print(f"OR: {odds_ratios['AirPollution1_Dummy_True']}")
    print(f"95% CI: ({conf_odds.loc['AirPollution1_Dummy_True', 0]}, {conf_odds.loc['AirPollution1_Dummy_True', 1]})")


Model one

In [ ]:
# Loop over each season
for season, df_season in Air_db.groupby('season'):
    print(f"\n\n####### Results for Season: {season} #######\n")
    ###### Ensure y is numeric
    y = pd.to_numeric(df_season['Birthweight_cate'], errors='coerce')
    #y = pd.to_numeric(df_season['PretermGrowth_cate'], errors='coerce')
    #y = pd.to_numeric(df_season['FulltermGrowth_cate'], errors='coerce')

    ###### Dummy encode binary categorical variables
    X_cat = pd.get_dummies(df_season[['AirPollution1_Dummy', 'sex']], drop_first=True)
    #X_cat = pd.get_dummies(df_season[['AirPollution2_Dummy', 'sex']], drop_first=True)
    #X_cat = pd.get_dummies(df_season[['AirPollution3_Dummy', 'sex']], drop_first=True)

    ###### Continuous variables
    X_cont = df_season[['age', 'BMI']]

    ###### Combine predictors
    X = pd.concat([X_cat, X_cont], axis=1)

    ###### Convert all to float (fix for bools)
    X = X.astype(float)

    ###### Fit the model
    from statsmodels.miscmodels.ordinal_model import OrderedModel
    model = OrderedModel(y, X, distr='logit', missing='drop')
    try:
        result = model.fit(method='bfgs', maxiter=1000, disp=False)
        print(result.summary())
    except Exception as e:
        print(f"Model fitting failed for season {season}: {e}")
        
    ###### Odds Ratio
    coef = result.params
    odds_ratios = np.exp(coef)
    conf = result.conf_int()
    conf_odds = np.exp(conf)
    print("\nOdds Ratio and 95% CI for AirPollution1_Dummy_True:")
    print(f"OR: {odds_ratios['AirPollution1_Dummy_True']}")
    print(f"95% CI: ({conf_odds.loc['AirPollution1_Dummy_True', 0]}, {conf_odds.loc['AirPollution1_Dummy_True', 1]})")

Model two

In [ ]:
# Loop over each season
for season, df_season in Air_db.groupby('season'):
    print(f"\n\n####### Results for Season: {season} #######\n")

    ###### Ensure y is numeric
    y = pd.to_numeric(df_season['Birthweight_cate'], errors='coerce')
    #y = pd.to_numeric(df_season['PretermGrowth_cate'], errors='coerce')
    #y = pd.to_numeric(df_season['FulltermGrowth_cate'], errors='coerce')

    ###### Dummy encode binary categorical variables
    X_cat = pd.get_dummies(df_season[['AirPollution1_Dummy', 'sex']], drop_first=True)
    #X_cat = pd.get_dummies(df_season[['AirPollution2_Dummy', 'sex']], drop_first=True)
    #X_cat = pd.get_dummies(df_season[['AirPollution3_Dummy', 'sex']], drop_first=True)

    ###### Other variables
    X_oth = df_season[['age', 'BMI','SocioEconomic','Met_pha1_zscore','ethnicities']]
    #X_oth = df_season[['age', 'BMI','SocioEconomic','Met_pha2_zscore','ethnicities']]
    #X_oth = df_season[['age', 'BMI','SocioEconomic','Met_pha3_zscore','ethnicities']]

    ###### Combine predictors
    X = pd.concat([X_cat, X_oth], axis=1)

    ###### Convert all to float (fix for bools)
    X = X.astype(float)

    ###### Fit the model
    from statsmodels.miscmodels.ordinal_model import OrderedModel
    model = OrderedModel(y, X, distr='logit', missing='drop')
    try:
        result = model.fit(method='bfgs', maxiter=1000, disp=False)
        print(result.summary())
    except Exception as e:
        print(f"Model fitting failed for season {season}: {e}")
        
    ###### Odds Ratio
    coef = result.params
    odds_ratios = np.exp(coef)
    conf = result.conf_int()
    conf_odds = np.exp(conf)
    print("\nOdds Ratio and 95% CI for AirPollution1_Dummy_True:")
    print(f"OR: {odds_ratios['AirPollution1_Dummy_True']}")
    print(f"95% CI: ({conf_odds.loc['AirPollution1_Dummy_True', 0]}, {conf_odds.loc['AirPollution1_Dummy_True', 1]})")

Model Three

In [ ]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

# Loop over each season
for season, df_season in Air_db.groupby('season'):

    print(f"\n\n####### Results for Season: {season} #######\n")

    ###### Ensure y is numeric
    y = pd.to_numeric(df_season['Birthweight_cate'], errors='coerce')
    #y = pd.to_numeric(df_season['PretermGrowth_cate'], errors='coerce')
    #y = pd.to_numeric(df_season['FulltermGrowth_cate'], errors='coerce')
    
    ###### Dummy encode binary categorical variables
    X_cat = pd.get_dummies(df_season[['AirPollution1_Dummy', 'sex', 'ApparentAbnormalitiesInBewBornTime_01','EmergencyorElective01']], drop_first=True)
    #X_cat = pd.get_dummies(df_season[['AirPollution2_Dummy', 'sex', 'ApparentAbnormalitiesInBewBornTime_01','EmergencyorElective01']], drop_first=True)
    #X_cat = pd.get_dummies(df_season[['AirPollution3_Dummy', 'sex', 'ApparentAbnormalitiesInBewBornTime_01','EmergencyorElective01']], drop_first=True)

    ###### Other variables
    X_oth = df_season[['age', 'BMI', 'SocioEconomic', 'Met_pha1_zscore', 'ethnicities']]
    #X_oth = df_season[['age', 'BMI', 'SocioEconomic', 'Met_pha2_zscore', 'ethnicities']]
    #X_oth = df_season[['age', 'BMI', 'SocioEconomic', 'Met_pha3_zscore', 'ethnicities']]

    ###### Combine predictors
    X = pd.concat([X_cat, X_oth], axis=1)

    ###### Convert all to float (fix for bools)
    X = X.astype(float)

    ###### Fit the model
    model = OrderedModel(y, X, distr='logit', missing='drop')
    
    try:
        result = model.fit(method='bfgs', maxiter=1000, disp=False)
        print(result.summary())
    except Exception as e:
        print(f"Model fitting failed for season {season}: {e}")
        
    ###### Odds Ratio
    coef = result.params
    odds_ratios = np.exp(coef)
    conf = result.conf_int()
    conf_odds = np.exp(conf)
    print("\nOdds Ratio and 95% CI for AirPollution1_Dummy_True:")
    print(f"OR: {odds_ratios['AirPollution1_Dummy_True']}")
    print(f"95% CI: ({conf_odds.loc['AirPollution1_Dummy_True', 0]}, {conf_odds.loc['AirPollution1_Dummy_True', 1]})")